In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder\
      .master("local")\
.config("spark.sql.adaptive.enabled",False)\
.config('spark.sql.autobroadcastjointhreshold',-1)\
.config('spark.sql.adaptive.coalescePartitions.enabled',False).getOrCreate()

In [3]:
spark

In [4]:
df = spark.read.format("parquet").load(r'C:\Users\Krishna\data\accounts_parquet\accounts_parquet')

In [5]:
df.show(4,False)

+--------+--------------+-------------+----------+---------+-------------------+----------+-----+-------+------------+-------------+-------------+
|acct_num|acct_create_dt|acct_close_dt|first_name|last_name|address            |city      |state|zipcode|phone_number|created      |modified     |
+--------+--------------+-------------+----------+---------+-------------------+----------+-----+-------+------------+-------------+-------------+
|32441   |1341603329000 |null         |Peter     |Zachary  |1891 Jessie Street |Long Beach|CA   |90743  |5628719002  |1395174646000|1395174646000|
|32442   |1345273687000 |null         |Duane     |Ruiz     |1023 Simpson Street|Pasadena  |CA   |91051  |6262472463  |1395174646000|1395174646000|
|32443   |1347189332000 |null         |Desiree   |Beall    |4830 Davis Place   |Reno      |NV   |89468  |7758006922  |1395174646000|1395174646000|
|32444   |1353649861000 |null         |Molly     |Pinder   |708 Locust Court   |Las Vegas |NV   |89103  |7024413066  |

In [6]:
df.count()

97320

In [7]:
df.explain()

== Physical Plan ==
*(1) ColumnarToRow
+- FileScan parquet [acct_num#0,acct_create_dt#1L,acct_close_dt#2L,first_name#3,last_name#4,address#5,city#6,state#7,zipcode#8,phone_number#9,created#10L,modified#11L] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/C:/Users/Krishna/data/accounts_parquet/accounts_parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<acct_num:int,acct_create_dt:bigint,acct_close_dt:bigint,first_name:string,last_name:string...




In [6]:
from pyspark.sql.functions import *

In [7]:
df1 = (df.filter(col('state')=='CA').withColumn('fn',upper('first_name'))\
                                               .withColumn('ln',upper('last_name'))\
        .select('fn','ln','state','city','zipcode'))

In [8]:
#noop simulates the write but no writing of data
df1.write.format('noop').mode('append').save(r'C:\Users\Krishna\data\aug2025')

In [9]:
ppl = spark.read.format('json').load(r'C:\Users\Krishna\data\people.json')

In [25]:
spark.conf.set("spark.sql.adaptive.enabled",False)
spark.conf.set('spark.sql.autobroadcastjointhreshold',-1)


In [18]:
ppl.join(df,ppl.pcode==df.zipcode).write.format('noop').mode('append').save(r'C:\Users\Krishna\data\aug2025')

In [13]:
ppl.join(df,ppl.pcode==df.zipcode).explain(False)

== Physical Plan ==
*(2) BroadcastHashJoin [pcode#123], [zipcode#8], Inner, BuildLeft, false
:- BroadcastExchange HashedRelationBroadcastMode(List(input[2, string, false]),false), [plan_id=158]
:  +- *(1) Filter isnotnull(pcode#123)
:     +- FileScan json [age#121L,name#122,pcode#123] Batched: false, DataFilters: [isnotnull(pcode#123)], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/C:/Users/Krishna/data/people.json], PartitionFilters: [], PushedFilters: [IsNotNull(pcode)], ReadSchema: struct<age:bigint,name:string,pcode:string>
+- *(2) Filter isnotnull(zipcode#8)
   +- *(2) ColumnarToRow
      +- FileScan parquet [acct_num#0,acct_create_dt#1L,acct_close_dt#2L,first_name#3,last_name#4,address#5,city#6,state#7,zipcode#8,phone_number#9,created#10L,modified#11L] Batched: true, DataFilters: [isnotnull(zipcode#8)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/C:/Users/Krishna/data/accounts_parquet/accounts_parquet], PartitionFilters: [], PushedFilters: [IsNotNull(zi

In [15]:
spark.conf.get("spark.sql.adaptive.enabled",'False')


'False'

In [20]:
spark.conf.set('spark.sql.cbo.enabled',True)

In [21]:
df.groupby('city').count().show()

+-------------+-----+
|         city|count|
+-------------+-----+
|      Phoenix| 3142|
|    Palo Alto| 1372|
|      Anaheim| 1182|
|      Oakland| 2718|
|  Bakersfield| 2290|
|         Bend| 1139|
|  Los Angeles| 2239|
|    Flagstaff|  804|
|   Sacramento| 5318|
|    Pendleton| 1096|
|     Industry| 1115|
|San Francisco| 2620|
|    San Diego| 3478|
|      Medford| 1116|
|   Santa Rosa| 2555|
|     Portland| 3432|
|      Redding| 1223|
|      Burbank| 1129|
|  Carson City|  996|
|     Van Nuys| 2243|
+-------------+-----+
only showing top 20 rows



In [22]:
df.printSchema()

root
 |-- acct_num: integer (nullable = true)
 |-- acct_create_dt: long (nullable = true)
 |-- acct_close_dt: long (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- created: long (nullable = true)
 |-- modified: long (nullable = true)



In [25]:
df.select('acct_num').distinct().count()

97320

In [27]:
df.groupby('acct_num').agg(countDistinct('city').alias('cnt')).show()

+--------+---+
|acct_num|cnt|
+--------+---+
|    3749|  1|
|   37263|  1|
|   96853|  1|
|   10817|  1|
|   96393|  1|
|   62985|  1|
|   74904|  1|
|   53565|  1|
|   61051|  1|
|    7340|  1|
|   89574|  1|
|   38422|  1|
|    5156|  1|
|   11858|  1|
|   27484|  1|
|   92959|  1|
|   33722|  1|
|   45307|  1|
|   56110|  1|
|   18498|  1|
+--------+---+
only showing top 20 rows



In [29]:
df1 = spark.range(4,200,2)
df2=spark.range(2,200,4)

In [40]:
df3=df1.repartition(5)
df4=df2.repartition(7)

In [35]:
df2.rdd.getNumPartitions()

7

In [36]:
df1.printSchema()

root
 |-- id: long (nullable = false)



In [41]:
dfj = df3.join(df4,'id')

In [42]:
dfres= dfj.selectExpr('sum(id) as cnt')

In [43]:
dfres.show()

+----+
| cnt|
+----+
|4998|
+----+



In [44]:
dfres.explain()

== Physical Plan ==
*(4) HashAggregate(keys=[], functions=[sum(id#426L)])
+- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=681]
   +- *(3) HashAggregate(keys=[], functions=[partial_sum(id#426L)])
      +- *(3) Project [id#426L]
         +- *(3) BroadcastHashJoin [id#426L], [id#428L], Inner, BuildRight, false
            :- Exchange RoundRobinPartitioning(5), REPARTITION_BY_NUM, [plan_id=671]
            :  +- *(1) Range (4, 200, step=2, splits=1)
            +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [plan_id=675]
               +- Exchange RoundRobinPartitioning(7), REPARTITION_BY_NUM, [plan_id=674]
                  +- *(2) Range (2, 200, step=4, splits=1)




In [45]:
df3.union(df4).show()

+---+
| id|
+---+
|104|
| 26|
| 22|
|160|
| 74|
|122|
| 64|
|114|
|190|
|182|
| 56|
|188|
|118|
| 72|
|154|
| 36|
|158|
| 24|
| 48|
|174|
+---+
only showing top 20 rows

